# 05 — Evaluation & Paper Figures

Run the benchmark, compare against baselines, generate paper-ready tables and figures.

1. Compute all metrics
2. Generalization analysis (seen vs unseen devices)
3. Ablation studies
4. Generate comparison tables

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5)

## 5.1 — Metrics on Synthetic Results

Demonstrating the evaluation framework with synthetic episode data.
Replace with real results after training.

In [ ]:
from safedisassemble.evaluation.metrics.disassembly_metrics import (
    DisassemblyEvaluator, EpisodeResult
)

evaluator = DisassemblyEvaluator(component_values={
    'ram': 8.0, 'ssd': 10.0, 'battery': 5.0, 'fan': 2.0,
    'screw': 0.1, 'panel': 0.5, 'antenna': 1.0,
})

# Simulate results for "our method"
np.random.seed(42)
for i in range(50):
    is_seen = i < 25  # first 25 on laptop (seen), next 25 on router (unseen)
    device = 'laptop_v1' if is_seen else 'router_v1'
    total = 11 if is_seen else 10
    
    # Our method is better on seen devices
    success_prob = 0.85 if is_seen else 0.55
    success = np.random.random() < success_prob
    n_recovered = total if success else np.random.randint(2, total)
    
    components = ['screw', 'panel', 'battery', 'ram', 'ssd', 'fan'][:n_recovered]
    violations = [] if np.random.random() > 0.1 else [{'type': 'battery_puncture'}]
    
    evaluator.add_result(EpisodeResult(
        device_name=device,
        total_components=total,
        recovered_components=components,
        damaged_components=[],
        safety_violations=violations,
        total_steps=np.random.randint(80, 300),
        total_reward=np.random.uniform(-5, 15),
        success=success and not violations,
        plan_was_safe=np.random.random() > 0.05,
        battery_disconnected_first=np.random.random() > 0.15,
    ))

metrics = evaluator.compute(
    seen_devices={'laptop_v1'},
    unseen_devices={'router_v1'},
)

print(evaluator.format_table(metrics))

## 5.2 — Generalization Bar Chart

In [ ]:
# Simulated results for multiple methods
methods = ['Ours (Full)', 'No Safety', 'Flat VLA', 'Scripted', 'Random']
seen_rates =   [0.85, 0.80, 0.60, 0.95, 0.02]
unseen_rates = [0.55, 0.50, 0.30, 0.00, 0.01]
safety_rates = [0.08, 0.35, 0.20, 0.05, 0.50]

x = np.arange(len(methods))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width, seen_rates, width, label='Seen Devices', color='#2196F3')
bars2 = ax.bar(x, unseen_rates, width, label='Unseen Devices', color='#FF9800')
bars3 = ax.bar(x + width, safety_rates, width, label='Safety Violations', color='#f44336', alpha=0.7)

ax.set_ylabel('Rate', fontsize=13)
ax.set_title('EWasteBench: Method Comparison', fontsize=15)
ax.set_xticks(x)
ax.set_xticklabels(methods, fontsize=11)
ax.legend(fontsize=11)
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.0%}', xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords='offset points', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('../renders/benchmark_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 5.3 — Safety Ablation

In [ ]:
# Demonstrate the impact of the safety module
categories = ['Battery\nPunctures', 'PCB\nSnaps', 'Battery-First\nCompliance', 'Overall\nSafety']

with_safety =    [0.03, 0.05, 0.92, 0.90]
without_safety = [0.28, 0.18, 0.45, 0.52]

x = np.arange(len(categories))
width = 0.3

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, with_safety, width, label='With Safety Module', color='#4CAF50')
ax.bar(x + width/2, without_safety, width, label='Without Safety Module', color='#f44336')

ax.set_ylabel('Rate', fontsize=13)
ax.set_title('Ablation: Impact of Safety Constraint Module', fontsize=15)
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=11)
ax.legend(fontsize=12)
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../renders/safety_ablation.png', dpi=150, bbox_inches='tight')
plt.show()

## 5.4 — Component Recovery Analysis

In [ ]:
# Per-component recovery rates
components = ['Screws', 'Panels', 'Battery', 'RAM', 'SSD', 'Fan', 'Antenna']
recovery_rates = [0.95, 0.88, 0.92, 0.75, 0.82, 0.70, 0.65]
values = [0.1, 0.5, 5.0, 8.0, 10.0, 2.0, 1.0]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Recovery rate
colors = plt.cm.RdYlGn(np.array(recovery_rates))
axes[0].barh(components, recovery_rates, color=colors)
axes[0].set_xlabel('Recovery Rate')
axes[0].set_title('Per-Component Recovery Rate')
axes[0].set_xlim(0, 1.05)
for i, v in enumerate(recovery_rates):
    axes[0].text(v + 0.01, i, f'{v:.0%}', va='center')

# Economic value recovered
value_recovered = [r * v for r, v in zip(recovery_rates, values)]
axes[1].barh(components, value_recovered, color='#2196F3')
axes[1].barh(components, [(1-r)*v for r, v in zip(recovery_rates, values)],
             left=value_recovered, color='#f44336', alpha=0.3)
axes[1].set_xlabel('Economic Value (relative units)')
axes[1].set_title('Value Recovered vs Lost')
axes[1].legend(['Recovered', 'Lost'], loc='lower right')

plt.suptitle('Component-Level Analysis', fontsize=14)
plt.tight_layout()
plt.savefig('../renders/component_analysis.png', dpi=150, bbox_inches='tight')
plt.show()